In [20]:
import numpy as np
import pandas as pd

data1 = pd.read_excel(r'pcbi.1006752.s002.xlsx', sheet_name="Sheet1",index_col='Drug').drop('SMILES',axis=1)
data2 = pd.read_excel(r'583drugs39cellsynergy.xlsx')
drug_names = list(data1.index)
# # 合并drugA和drugB为一列作为索引
# 顺序
temp1 = [] # 分子指纹合并值 【【】，【】，【】，】
index_1 = [] # 对应的名字列表 【【】，【】，【】，】
# 逆序
temp2 = []
index_2 = []
for index,row in data2.iterrows():
    ct = list((np.array(data1.loc[row["DrugA"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugB"],:]))))
    temp1.append(ct)
    index_1.append((row["DrugA"],row["DrugB"]))
    ct = list((np.array(data1.loc[row["DrugB"],:])))
    ct.extend(list((np.array(data1.loc[row["DrugA"],:]))))
    temp2.append(ct)
    index_2.append((row["DrugB"],row["DrugA"]))


# 两组数据的tanimoto计算
def tanimoto(p,q):
    tep1=0
    tep2=0
    lenthp=int(len(p))
    for i in range(lenthp):
        a=p[i]
        b=q[i]
        if (a==1)|(b==1): # 并
            tep1=tep1+1
        if (a==1)&(b==1): # 交
            tep2=tep2+1
    c=round((tep2 / tep1),4)      #取值4位数
    return c

# 相互之间做一次tanimoto
temp=[]
Tanimoto_index_dist = {} # {顺序药物对：(实际药物名字1，实际药物名字2，值)}
for i,j,i_n,j_n in zip(temp1,temp2,index_1,index_2):
    tmp=[]
    a = np.array(i) #取出一分子指纹 转换列表为向量
    b = np.array(j)
    for k,m,k_n,m_n in zip(temp1,temp2,index_1,index_2): # 遍历每一个药物对的分子指纹
        c = np.array(k)
        d = np.array(m)
        tp1 = tanimoto(a,c)
        tp2 = tanimoto(a,d)
        tp3 = tanimoto(b,c)
        tp4 = tanimoto(b,d)
        tps = [tp1, tp2, tp3, tp4]
        index_list = [(i_n,k_n,tp1),(i_n,m_n,tp2),(j_n,k_n,tp3),(j_n,m_n,tp4)]
        index_location = tps.index(max(tps))
        tmp.append(max(tps))
        Tanimoto_index_dist[f"{i_n},{k_n}"] = index_list[index_location]
    temp.append(tmp)
Tanimoto=pd.DataFrame(temp,columns=index_1,index=index_1) # 数据框 dataframe

/root/miniconda3/envs/jupyter-env/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [21]:
# 获取栏目数组
ttpp = []
for i in list(Tanimoto.columns):
    ttpp.append(str(i)[1:-1]) # 去除首尾括号的药物对名字
data2 = pd.DataFrame(data2.iloc[:,2:].T.values, columns=list(ttpp)) # 药物对协同得分
columns = data2.columns # 修改后的药物对名字


# 删去对应行后的字典
dict_lines = {} # {键1：值1} 谷本系数字典
count = 0
ct = 0
for i in Tanimoto.index:
    y = list(Tanimoto.iloc[:,ct]) # 每一列谷本系数 [:,1] 第一列的每一行 实际就是第一列
    y.pop(count)
    count += 1
    dict_lines[columns[ct]] = y # 谷本系数药物对的名字做键1，y是值1
    ct += 1

# dict_lines["'METHOTREXATE', 'TEMOZOLOMIDE'"]

In [22]:
# 多少高相似
coum=5

# 分离x y数据构造字典
dict_columns = {}  # {a1:data,a2:data} 药物对名字-对应数据
for i in columns:

    # b 要预测协同得分那一列
    y = data2.loc[:, i]

    # A 权重
    temp = []
    for j in columns:
        if j != i:
            temp.append(j)
    m = data2.loc[:, temp[0]]
    tp = [m]
    for j in temp[1:]:
        tp.append(data2.loc[:, j])
    x = pd.DataFrame(tp).T  # 其他所有的列

    # 高相似的    取高相似的药物对名字max1和对应的值max5
    tp = []
    for j in columns:
        if i != j:
            tp.append(j)
    df = pd.DataFrame(dict_lines[i], index=tp)
    df.sort_values(by=0, inplace=True, ascending=True)  # 从低到高排序
    max_l = df.index[-coum:]  # 倒着取高相似的5个的名字 排序还是从低到高
    tp = []
    for j in max_l:
        tp.append(data2.loc[:, j])
    max_5x = pd.DataFrame(tp).T  # 值

    # 存储 对应数据
    data = {
        "x": x,  # 其他所有的列 数据表，含药物对名字
        "y": y,  # 要预测协同得分的那一列 含药物对名字
        # 下面两个是对应的
        "max_relevance": max_l,  # 高相似名字
        "max_5x": max_5x  # 前多少高相似邻居对应的列
    }
    dict_columns[i] = data

dict_columns_split = {} # 高相似字典
for i in dict_columns: # 第几列 i 是药物对名字
    count = 0 # 第几行 int
    for j in list(dict_columns[i]['y'].index): # 35个索引 0~34 数字（字符串-str） 【0，1，2，3.。。。】
        dict_columns_split[f'{i},{j}']={} # 空字典 键-空字典
        dict_columns_split[f'{i},{j}']['y_line']=dict_columns[i]["y"][count]
        dict_columns_split[f'{i},{j}']['max_x_line']=dict_columns[i]["max_5x"].iloc[count,:].values
        # print(dict_columns[i]["max_5x"].iloc[count,:].values)
        count += 1

In [23]:
dict_columns_split["'SUNITINIB', 'MK-4827',32"]['max_x_line']

array([11.72948608, -2.06607527, 27.37080776, 14.44972614,  5.95041914])

In [24]:
# 删去对应行后的字典
dict_high_lines = {} #高相似谷本系数字典
count = 0
cct = 0
for i in Tanimoto.index: # 循环遍历谷本系数名字
    y = list(Tanimoto.iloc[:,cct])
    y.pop(count) # 去除相同药物对的谷本系数
    count += 1
    dict_high_lines[str(i)[1:-1]] = y # 去除药物对首尾括号做键，去除后剩余的谷本系数列做值

    tp = []
    for j in columns: #来源于协同得分名字和来源于谷本系数名字比较，不同的就存储
        if str(i)[1:-1] != j:
            tp.append(j)
    df = pd.DataFrame(dict_high_lines[str(i)[1:-1]], index=tp) # 除去1的数据框 582*1
    df.sort_values(by=0 , inplace=True, ascending=True) # 排序 低到高

    max_l = df.index[-coum:]  # 取最后5高相似
    tp = [] # 5个名字对应列的值
    for j in max_l:
        tp.append(Tanimoto.iloc[list(columns).index(j), :])
    max_5y = list(pd.DataFrame(tp).T.iloc[[list(columns).index(str(i)[1:-1])]].values)[0]

    data = {
        "y": y,
        "max_relevance": max_l,
        "high_5": max_5y
    }
    dict_high_lines[str(i)[1:-1]] = data
    cct += 1

dict_high_lines_split = {}
for i in dict_high_lines: # i 是目标药物对名字
    count = 0
    for j in dict_high_lines[i]['max_relevance']: # 遍历高相似的名字
        dict_high_lines_split[f'{i},{j}']=dict_high_lines[i]['high_5'][count]
        count += 1

TTT = [] # 值列表
for i in dict_high_lines_split:
    TTT.append(dict_high_lines_split[i])

In [25]:
Name = []
a = 1
lin = np.linspace(0.015,5,998)
for linename in columns:
    PCC = 0
    MSE = 10000
    result = []
    for t_coum in range(2, coum+1, 1):    
        #求MSE
        alpha = []
        result_mse = []
        mse_split = []
        for alp in lin:
            r1 = []
            ct = 0
            for elem in range(39):
                pp = []
                for j in TTT[(coum * a - t_coum):(coum * a)]: 
                    pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))#改函数
                r1.append(np.sum((pp/np.sum(pp)) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]) -
                        dict_columns_split[f"{linename},{elem}"]['y_line'])
            mse_split.append(np.mean(np.square(r1)))
        result_mse.append(np.min(mse_split))
        alpha.append(lin[mse_split.index(np.min(mse_split))])
        mse = result_mse[0]
        #求PCC
        pre = []
        alp = alpha[0]
        for elem in range(39):
            pp = []
            for j in TTT[(coum * a - t_coum):(coum * a)]: 
                pp.append(np.exp(-pow(1 - j, 2) / (2 * pow(alp, 2))))  # 改函数
            pre.append(np.sum(pp / np.sum(pp) * dict_columns_split[f"{linename},{elem}"]['max_x_line'][-t_coum:]))
        pcc = np.corrcoef(pre, data2[linename])[0, 1]
        if mse < MSE:
            Name_coum = t_coum
            Name_mse = mse
            Name_pcc = pcc
            Name_alp = alp
            MSE = mse
        elif MSE == 10000:
            Name_coum, Name_mse, Name_pcc, Name_alp, MSE  = t_coum, mse, pcc, alp, mse
        print("name:{},coum:{:0d},alp={},mse={},pcc={}".format(linename, t_coum, alp, mse, pcc))
    a = a+1
    Name.append([linename, Name_coum, Name_alp, Name_mse, Name_pcc])
pd.DataFrame(Name).to_csv('mse_origin.csv', header=None, index=False)

name:'MK-5108', 'SORAFENIB',coum:2,alp=5.0,mse=132.15436142637068,pcc=0.35595664272467803
name:'MK-5108', 'SORAFENIB',coum:3,alp=5.0,mse=152.84542458258036,pcc=0.4188429174891831
name:'MK-5108', 'SORAFENIB',coum:4,alp=5.0,mse=107.19339959176178,pcc=0.4430126351339127
name:'MK-5108', 'SORAFENIB',coum:5,alp=0.885,mse=95.80863243819635,pcc=0.4295253927558233
name:'VINORELBINE', 'SUNITINIB',coum:2,alp=5.0,mse=333.2227263110906,pcc=0.43646807853676767
name:'VINORELBINE', 'SUNITINIB',coum:3,alp=5.0,mse=222.7652339115941,pcc=0.46011689802334393
name:'VINORELBINE', 'SUNITINIB',coum:4,alp=5.0,mse=161.7999954751967,pcc=0.45939502771584234
name:'VINORELBINE', 'SUNITINIB',coum:5,alp=5.0,mse=136.62084192560548,pcc=0.4510691464268737
name:'SUNITINIB', 'MK-8776',coum:2,alp=0.14500000000000002,mse=142.9122643268362,pcc=0.2865022219342793
name:'SUNITINIB', 'MK-8776',coum:3,alp=0.18,mse=104.08063950849004,pcc=0.5673812513369484
name:'SUNITINIB', 'MK-8776',coum:4,alp=0.52,mse=93.90563151922629,pcc=0.6038